# How to add custom biosphere flows

## Expand on existing Characterization method

In [61]:
import bw2data as bd
import importlib


# import the functions from the file
##
# Reload the module to pick up any changes (useful during development)
import add_new_cfs

importlib.reload(add_new_cfs)


from add_new_cfs import parse_new_flows_from_csv, parse_node_ids_and_cfs

In [62]:
# Set the project
bd.projects.set_current("PFAS_SCWO_US")

In [64]:
### First, create the new nodes
### and add them to a new database
new_flows = parse_new_flows_from_csv(r"tests\PFAS_CF_USEtox_adapted_EC20.csv")
# The new database name is meant to be the same that you put inthe csv file "new_database" column
new_db = bd.Database("PFAS_biosphere")
new_db.register()
new_db.write(new_flows)

100%|██████████| 10/10 [00:00<?, ?it/s]


13:47:45+0100 [info     ] Vacuuming database            


In [65]:
### Second, parse the new cfs
new_cfs = parse_node_ids_and_cfs(r"tests\PFAS_CF_USEtox_adapted_EC20.csv")

In [58]:
## Find the available methods
"""
choose impact method and create list of methods
"""

ia = "USEtox v2.13 PFAS adapted, midpoint no LT"  # change if needed
ia_category = []
for m in bd.methods:
    try:
        if m[0] == ia:  # looks for exact match of method family name
            # m[0] refers to the first element in the list of tuples - (method family name, impact category, method indicator)
            ia_category.append(m)
    except Exception as e:
        print(f"Error processing method {m}: {e}")

if ia_category:
    print(f"Impact assessment methods with {ia} as method family:")
    for i, method in enumerate(ia_category, start=0):
        print(f"{i}. {method}")
else:
    print(f"No methods found with '{ia}' in the method family name.")
    print("Available method families:")
    # Show first few method families to help debug
    unique_families = set()
    for m in bd.methods:
        try:
            unique_families.add(m[0])
        except:
            pass
    for i, family in enumerate(sorted(unique_families), 1):
        print(f"{i}. {family}")
    print("... (showing first 10)")

Impact assessment methods with USEtox v2.13 PFAS adapted, midpoint no LT as method family:
0. ('USEtox v2.13 PFAS adapted, midpoint no LT', 'ecotoxicity: freshwater no LT', 'comparative toxic unit for ecosystems (CTUe) no LT')


In [66]:
### Third, load the cfs of another method
other_method = bd.Method(
    (
        "USEtox v2.13, midpoint no LT",
        "ecotoxicity: freshwater no LT",
        "comparative toxic unit for ecosystems (CTUe) no LT",
    )
)

other_method_cfs = other_method.load()

new_method = bd.Method(
    (
        "USEtox v2.13 PFAS adapted, midpoint no LT",
        "ecotoxicity: freshwater no LT",
        "comparative toxic unit for ecosystems (CTUe) no LT",
    )
)
new_method.register()
all_cfs = other_method_cfs.extend(new_cfs)
new_method.write(new_cfs)

In [67]:
### Verification that it works can be done with:
new_method = bd.Method(
    (
        "USEtox v2.13 PFAS adapted, midpoint no LT",
        "ecotoxicity: freshwater no LT",
        "comparative toxic unit for ecosystems (CTUe) no LT",
    )
)
new_method.load()

# for a in new_method:
# print(a)
bd.get_node(code="PFNA")

'Perfluorononanoic acid' (kg, None, ('water', 'surface water', 'freshwater'))

In [ ]:
# check if db is there ;)
bd.projects.set_current("PFAS_SCWO_US")
bd.databases

Databases dictionary with 5 object(s):
	PFAS degradation
	PFAS_biosphere
	biosphere-3.11
	ei11_cutoff_remind_eu_ssp2_35_50
	ei311cutoff

### Error Spotting

In [ ]:
import csv

In [21]:
# Alternative: If you need to keep the reader open for iteration
# Open the file and create the reader
# Use 'utf-8-sig' encoding to automatically strip the BOM (Byte Order Mark) if present
# The BOM (\ufeff) is added by some programs like Excel and causes the first column name to be misread
f = open(r"tests\PFAS_CF_USEtox_adapted_EC20.csv", "r", encoding="utf-8-sig")
reader = csv.DictReader(f)

# Access column headings
print("Column headings:", reader.fieldnames)
# Or assign to a variable
column_headings = reader.fieldnames

Column headings: ['new_database', 'flow_name', 'code', 'unit', 'CAS number', 'categories', 'cf']


In [ ]:
import pandas as pd

file = pd.read_csv(r"tests\PFAS_CF_USEtox_adapted_EC20.csv")

In [17]:
file.head()

,new_database,flow_name,code,unit,CAS number,categories,cf
0,PFAS_biosphere,Perfluorononanoic acid,PFNA,kg,375-95-1,water::surface water::freshwater,828.583876
1,PFAS_biosphere,Perfluorooctanoic acid,PFOA,kg,335-67-1,water::surface water::freshwater,894.185577
2,PFAS_biosphere,Perfluoroheptanoic acid,PFHpA,kg,375-85-9,water::surface water::freshwater,43.618676
3,PFAS_biosphere,Perfluorohexanoic acid,PFHxA,kg,307-24-4,water::surface water::freshwater,85.239291
4,PFAS_biosphere,Perfluoropentanoic acid,PFPeA,kg,2706-90-3,water::surface water::freshwater,213.519594
